In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pathlib import Path

#import api_key
#from api_keys import api_keys

In [ ]:
#Createing path to CSV files
Flu_path = Path("../Resources/ILINet.csv")
Temp_path = Path("../Resources/TempByState_1855_2013.csv")


In [9]:
#Create data frame
Flu_DF = pd.read_csv(Flu_path)
Temp_DF = pd.read_csv(Temp_path)

Flu_DF.head()
Temp_DF.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


In [10]:
Flu_DF.head()

PERCENTAGE OF VISITS FOR INFLUENZA-LIKE-ILLNESS REPORTED BY SENTINEL PROVIDERS
REGION TYPE REGION   YEAR WEEK % WEIGHTED ILI %UNWEIGHTED ILI AGE 0-4 AGE 25-49 AGE 25-64 AGE 5-24 AGE 50-64 AGE 65 ILITOTAL NUM. OF PROVIDERS                                     TOTAL PATIENTS                            
HHS Regions Region 4 2016 40   1.22375        1.36456         637     369       NaN       801      115       84     2006     306                                                           147007                            
                          41   1.23174        1.45288         714     399       NaN       769      153       96     2131     317                                                           146674                            
                          42   1.2768         1.51243         782     414       NaN       898      185       93     2372     323                                                           156834                            
                          43   1.38032        1.66763         789     532       NaN       994      198       109    2622     325                                                           157229

In [11]:
temp_df_us = pd.DataFrame(Temp_DF.loc[Temp_DF["Country"] == "United States",:])

temp_df_us.head()


,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
7458,1743-11-01,10.722,2.898,Alabama,United States
7459,1743-12-01,NaN,NaN,Alabama,United States
7460,1744-01-01,NaN,NaN,Alabama,United States
7461,1744-02-01,NaN,NaN,Alabama,United States
7462,1744-03-01,NaN,NaN,Alabama,United States


In [12]:
#Dropping NaN
temp_df_us = temp_df_us.dropna(subset=['AverageTemperature', 'AverageTemperatureUncertainty'])

temp_df_us.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
7458,1743-11-01,10.722,2.898,Alabama,United States
7463,1744-04-01,19.075,2.902,Alabama,United States
7464,1744-05-01,21.197,2.844,Alabama,United States
7465,1744-06-01,25.290,2.879,Alabama,United States
7466,1744-07-01,26.420,2.841,Alabama,United States


In [13]:
#scrubbing data
temp_df_us =  temp_df_us.loc[:,["dt","AverageTemperature","State"]]

temp_df_us.head()

,dt,AverageTemperature,State
7458,1743-11-01,10.722,Alabama
7463,1744-04-01,19.075,Alabama
7464,1744-05-01,21.197,Alabama
7465,1744-06-01,25.290,Alabama
7466,1744-07-01,26.420,Alabama


In [14]:
#Convert Celsius to  Farenheit f=(c*9/5)+32

temp_convert = temp_df_us.copy()

temp_convert['AverageTemperature'] = (temp_df_us['AverageTemperature'] * 9/5) + 32

temp_convert.head()

,dt,AverageTemperature,State
7458,1743-11-01,51.2996,Alabama
7463,1744-04-01,66.3350,Alabama
7464,1744-05-01,70.1546,Alabama
7465,1744-06-01,77.5220,Alabama
7466,1744-07-01,79.5560,Alabama


In [15]:
# Define the Temperature threshold groupby
high_temp_threshold = 85
low_temp_threshold = 65

# Categorize temperatures
def categorize_temperature(temp):
    if temp > high_temp_threshold:
        return 'High Temp'
    elif temp < low_temp_threshold:
        return 'Low Temp'
    else:
        return 'Medium Temp'
    
# Apply the categorization function to createa new column
temp_convert['Temperature Category'] = temp_convert['AverageTemperature'].apply(categorize_temperature)

temp_convert.head()

,dt,AverageTemperature,State,Temperature Category
7458,1743-11-01,51.2996,Alabama,Low Temp
7463,1744-04-01,66.3350,Alabama,Medium Temp
7464,1744-05-01,70.1546,Alabama,Medium Temp
7465,1744-06-01,77.5220,Alabama,Medium Temp
7466,1744-07-01,79.5560,Alabama,Medium Temp
